In [0]:
import sys
workspace = dbutils.widgets.get('workspace')
sys.path.insert(0, workspace)

from libs.ingestao_silver import ingestao_silver

from pyspark.sql.functions import col, cast, trim, when, to_date, upper, regexp_replace

In [0]:
catalog = 'projeto_cnpj'
table = 'empresas'
df = spark.table(f"{catalog}.bronze.{table}")

df= (df
    .withColumn('razao_social', trim(upper(col('razao_social'))))
    .withColumn('natureza_juridica', col('natureza_juridica').cast('int'))
    .withColumn('qualificacao_responsavel', regexp_replace(col('qualificacao_responsavel'), r'[^0-9.]', '').cast('int'))
    .withColumn('capital_social', regexp_replace(col('capital_social'), r'[^0-9.]', '.').cast('float'))
    .withColumn('data_ingestao', col('data_ingestao').cast('date'))
    .withColumn('porte',
        when(trim(col('porte')) == '05', 'Demais')
        .when(trim(col('porte')) == '01', 'Micro Empresa')
        .when(trim(col('porte')) == '03', 'Empresa Pequeno Porte')
        .when(trim(col('porte')) == '00', 'Não Informado').otherwise('Outros')
    )
)

In [0]:
ingestao_silver(
    spark = spark,
    df = df,
    catalog = catalog,
    table = table
)